<a href="https://colab.research.google.com/github/JIng-Yu-Huang/generative-AI_hw/blob/main/0429%E6%89%93%E9%80%A0Stable_Diffusion%E7%9A%84WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 打造 Stable Diffusion 的 WebUI

### 1. 安裝必要套件

In [ ]:
!pip install diffusers transformers accelerate safetensors huggingface_hub gradio --upgrade

In [ ]:
# from google.colab import userdata

# hf_token = userdata.get("HuggingFace")
# login(token=hf_token)

In [ ]:
from diffusers import StableDiffusionPipeline, UniPCMultistepScheduler
import torch
import gc
import matplotlib.pyplot as plt
import gradio as gr
import random

### 2. 指定並讀入模型

In [ ]:
model_name = "Lykon/dreamshaper-8"

注意有可能要停用 `use_safetensors=True`。

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

In [ ]:
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

### 3. 生成的函式

In [ ]:
def generate_images(prompt, use_enhance, enhance_text, use_negative, negative_text,
                    use_custom_seed, custom_seed, height, width, steps, num_images):

    height = int(height)
    width = int(width)

    if height % 8 != 0 or width % 8 != 0:
        raise ValueError("高度和寬度必須是8的倍數！")

    if use_custom_seed:
        base_seed = int(custom_seed)
    else:
        base_seed = random.randint(0, 2**32 - 1)

    seeds = [base_seed + i for i in range(num_images)]

    prompts = []
    negative_prompts = []
    generators = []

    final_prompt = prompt
    if use_enhance and enhance_text:
        final_prompt = prompt + ", " + enhance_text

    final_negative = negative_text if use_negative else None

    for seed in seeds:
        g = torch.Generator("cuda").manual_seed(seed)
        generators.append(g)
        prompts.append(final_prompt)
        negative_prompts.append(final_negative)

    gc.collect()
    torch.cuda.empty_cache()

    images = []
    for i in range(num_images):
        with torch.no_grad():
            image = pipe(
                prompt=prompts[i],
                negative_prompt=negative_prompts[i] if final_negative else None,
                height=height,
                width=width,
                num_inference_steps=steps,
                guidance_scale=7.5,
                generator=generators[i]
            ).images[0]
            images.append(image)

    return images, f"使用的 random seeds: {seeds}"

### 4. 打造 Gradio Web App

In [ ]:
default_enhance = (
    # 高品質增強提示詞
    "8k, RAW photo, best quality, extremely detailed, perfect lighting, soft skin, "
    "sharp eyes, beautiful face, ultra-realistic"
    # 8K、RAW 照片、最佳品質、極度細緻、完美打光、柔嫩皮膚、銳利眼神、美麗臉龐、超寫實風格
)

default_negative = (
    # 避免出現的內容（負面提示詞）
    "bad anatomy, blurry, disfigured, poorly drawn hands, extra fingers, fused fingers, "
    "mutated hands, deformed, lowres, jpeg artifacts, watermark, long neck, cloned face, ugly"
    # 解剖錯誤、模糊、畸形、手部畫錯、多手指、手指黏在一起、變異手、扭曲、低畫質、壓縮痕、水印、脖子過長、臉部複製、醜
)
# 新增：根據風格自動設定 enhance 與 negative
def apply_style(style):
    if style == "🧩 預設風格":
        return (
            "8k, RAW photo, best quality, extremely detailed, perfect lighting, soft skin, "
            "sharp eyes, beautiful face, ultra-realistic",
            "bad anatomy, blurry, disfigured, poorly drawn hands, extra fingers, fused fingers, "
            "mutated hands, deformed, lowres, jpeg artifacts, watermark, long neck, cloned face, ugly"
        )
    elif style == "🩰 可愛甜美風":
        return (
            "pastel colors, soft lighting, kawaii style, adorable face, light tones",
            "ugly, scary, deformed, creepy, dark lighting"
        )
    elif style == "📷 寫實精緻風":
        return (
            "8k, ultra realistic, perfect face, best quality, sharp focus",
            "lowres, bad hands, bad anatomy, jpeg artifacts"
        )
    elif style == "🎨 動漫插畫風":
        return (
            "anime style, cel shading, big eyes, colorful background, detailed lineart",
            "poorly drawn, sketchy, distorted, extra limbs"
        )
    return "", ""

with gr.Blocks(css="""
.gradio-container {background-color: #FFE4F0; padding: 20px;}
.gr-button {font-size: 18px; background: linear-gradient(to right, #FF9A9E, #FAD0C4); color: white;}
""") as demo:
    gr.Markdown("""
    # 💁‍♀️ DreamShaper 人物圖像生成器
    選好提示詞與參數，馬上創作出你夢想中的角色照片！
    """)

    with gr.Row():
        with gr.Column(scale=6):
            prompt = gr.Textbox(
                label="Prompt",
                placeholder="a cute girl sitting on the grass, eating basque burnt cheesecake and ice cream",
                lines=3
            )

            with gr.Row():
                use_enhance = gr.Checkbox(label="加強 Prompt 效果", value=True)
                enhance_text = gr.Textbox(label="加強內容", value=default_enhance)

            with gr.Row():
                use_negative = gr.Checkbox(label="加入負面提示詞 (Negative Prompt)", value=True)
                negative_text = gr.Textbox(label="Negative Prompt 內容", value=default_negative)

            # 🎨 新增風格選單與按鈕
            with gr.Row():
                style_selector = gr.Dropdown(
                    ["🧩 預設風格","🩰 可愛甜美風", "📷 寫實精緻風", "🎨 動漫插畫風"],
                    label="🎨 選擇風格並套用!",
                    value="🧩 預設風格"
                )
                style_btn = gr.Button("➡️ 套用風格設定")

            style_btn.click(fn=apply_style, inputs=style_selector, outputs=[enhance_text, negative_text])

            with gr.Row():
                use_custom_seed = gr.Checkbox(label="自訂 Random Seed", value=False)
                custom_seed = gr.Number(label="指定 seed (選填)", value=42)

            with gr.Row():
                height = gr.Dropdown(["512", "768", "1024"], label="圖片高度", value="768")
                width = gr.Dropdown(["512", "768", "1024"], label="圖片寬度", value="512")

            with gr.Row():
                steps = gr.Slider(10, 50, value=30, step=5, label="生成步數 (Steps)")
                num_images = gr.Slider(1, 4, step=1, value=1, label="生成張數")

            generate_btn = gr.Button("✨ 開始創作！")

        with gr.Column(scale=6):
            gallery = gr.Gallery(label="生成結果", columns=2, object_fit="contain", height="auto")
            seed_info = gr.Label(label="使用的 Random Seeds")

    generate_btn.click(
        fn=generate_images,
        inputs=[prompt, use_enhance, enhance_text, use_negative, negative_text,
                use_custom_seed, custom_seed, height, width, steps, num_images],
        outputs=[gallery, seed_info]
    )


In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    "Lykon/dreamshaper-8",
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

pipe.enable_attention_slicing()
result = pipe(
    prompt="a cute girl sitting in a flower field, holding a strawberry parfait, smiling at the camera",
    negative_prompt="ugly, bad anatomy, blurry",
    height=512,
    width=512,
    num_inference_steps=30
).images[0]
result.save("test.png")


In [ ]:
from diffusers import StableDiffusionPipeline, DDIMScheduler
import torch, gc

model_id = "Lykon/dreamshaper-8"

# ① 全 float16 載入
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

# ② 換成穩定的 DDIM 排程器（黑圖多半是 Scheduler 溢出）
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

pipe.enable_attention_slicing()  # 省顯存，可留可拿掉

gc.collect(); torch.cuda.empty_cache()

# ③ 測試產生一張
image = pipe(
    prompt="a cute girl sitting in a flower field, holding a strawberry parfait, smiling at the camera, pastel colors, soft lighting, ultra detailed, dreamy atmosphere",
    negative_prompt="ugly, bad anatomy, blurry",
    height=512,
    width=512,
    num_inference_steps=30
).images[0]

image


In [ ]:
demo.launch(share=True, debug=True)